In [18]:
import pandas as pd
import json
import re

# data = pd.read_csv("NEW_data_collection/data/web_data_with_questions.csv")

In [19]:
# avg_prompt_tokens = sum([json.loads(data.iloc[idx, 2])['usage']['prompt_tokens'] for idx in range(len(data))])/len(data)
# avg_completion_tokens = sum([json.loads(data.iloc[idx, 2])['usage']['completion_tokens'] for idx in range(len(data))])/len(data)

# avg_cost_per_page = 0.01 * (avg_prompt_tokens/1000) + 0.03 * (avg_completion_tokens/1000)
# print(avg_cost_per_page*7000)

In [20]:
# avg_cost_per_page = 0.001 * (avg_prompt_tokens/1000) + 0.002 * (avg_completion_tokens/1000)
# print(avg_cost_per_page*7000)

# Side by side gpt3.5 and gpt4 questions

In [21]:
import pandas as pd
import json

data3 = pd.read_csv("NEW_data_collection/data/GPT3.5 questions.csv")
data4 = pd.read_csv("NEW_data_collection/data/GPT4 questions.csv")



# data3 and data4 are almost the same just with different questions. 
# We will join on questions then view them side by side to see if there are any differences
combined = pd.merge(data3, data4, on=['url', 'data'], how='inner')

# GPT 4

SUPER easy to parse the output. UNLIKE GPT3.5

In [32]:
data4['cleaned_qa'] = [json.loads(data4.loc[idx, 'questions'])['choices'][0]['message']['content'].strip().replace("questions", "") for idx in range(len(data4))]
data4['cleaned_qa']

0     ```json\n[\n    {\n        "question": "How ca...
1     ```json\n[\n  {\n    "question": "How does SUN...
2     ```json\n[\n  {\n    "question": "How can I an...
3     ```json\n[\n  {\n    "question": "What kind of...
4     ```json\n[\n  {\n    "question": "What is the ...
5     ```json\n[\n    {\n        "question": "How ca...
6     ```json\n[\n    {\n        "question": "How ca...
7     ```json\n[\n  {\n    "question": "How can I ap...
8     ```json\n[\n    {\n        "question": "What c...
9     ```json\n[\n    {\n        "question": "How ca...
10    ```json\n[\n  {\n    "question": "How can I sc...
11    ```json\n[\n  {\n    "question": "How can I su...
12    ```json\n[\n    {\n        "question": "How ca...
13    ```json\n[\n  {\n    "question": "Where can I ...
14    ```json\n[\n  {\n    "question": "How can futu...
15    ```json\n[\n  {\n    "question": "How do I get...
16    ```json\n[\n  {\n    "question": "How can I le...
17    ```json\n[\n  {\n    "question": "How can 

In [36]:
data4['super_cleaned_qa'] = data4['cleaned_qa'].apply(lambda x: json.loads(x[7:-3]))

In [37]:
data4['super_cleaned_qa']

0     [{'question': 'How can I schedule an undergrad...
1     [{'question': 'How does SUNY Brockport foster ...
2     [{'question': 'How can I anonymously report a ...
3     [{'question': 'What kind of academic support d...
4     [{'question': 'What is the duration of the Fal...
5     [{'question': 'How can I develop my leadership...
6     [{'question': 'How can students benefit from u...
7     [{'question': 'How can I apply for scholarship...
8     [{'question': 'What criteria should I meet to ...
9     [{'question': 'How can I apply to SUNY Brockpo...
10    [{'question': 'How can I schedule a tour of th...
11    [{'question': 'How can I subscribe to The Port...
12    [{'question': 'How can I make an online donati...
13    [{'question': 'Where can I find the course cat...
14    [{'question': 'How can future students apply t...
15    [{'question': 'How do I get involved with mult...
16    [{'question': 'How can I learn more about SUNY...
17    [{'question': 'How can I apply for financi

# GPT 3.5

These are super hard responses to parse. It's responses are really all over the place. So much for "JSON mode"

In [22]:
data3['cleaned_qa'] = [json.loads(data3.loc[idx, 'questions'])['choices'][0]['message']['content'].strip().replace("questions", "") for idx in range(len(data3))]

In [23]:
data3['cleaned_qa'][4]

'{"": [\n  {"question": "What are the dates for the Fall Semester 2023?", "answer": "The Fall Semester 2023 dates have not been specified."},\n  {"question": "When is the Winter Session 2024?", "answer": "The date for the Winter Session 2024 has not been mentioned."},\n  {"question": "How long are the M.W.F. classes during the Fall Semester?", "answer": "M.W.F. classes during the Fall Semester are 50 minutes in length."},\n  {"question": "Can students request time off for religious beliefs?", "answer": "Yes, under section 224-A of the Education Law of the State of New York, absence from class for religious beliefs is excused."},\n  {"question": "How long are the T.R. classes during the Spring Semester?", "answer": "T.R. classes during the Spring Semester are 75 minutes in length."}\n]}'

In [24]:
print(data3['cleaned_qa'][22])

{"": [
{"question": "How can I report a bias related incident at SUNY Brockport?", "answer": "You can report a bias related incident at SUNY Brockport through their Bias Related Incident Reporting System."},
{"question": "Who should I contact for urgent facilities issues at SUNY Brockport?", "answer": "For urgent facilities issues at SUNY Brockport, you can directly call the Service Center at ext. 2408."},
{"question": "How do I report accidents or injuries or emergency spill responses at SUNY Brockport?", "answer": "To report accidents or injuries or emergency spill responses at SUNY Brockport, you can contact their Environmental Health & Safety department."},
{"question": "Who can I contact at SUNY Brockport for IT assistance?", "answer": "If you require IT assistance at SUNY Brockport, you can contact their IT Service Desk for help."},
{"question": "What is the process for appealing a grade at SUNY Brockport?", "answer": "SUNY Brockport has a grade appeals process, and students have

In [29]:
def get_json_qa(value):
    jsonqa = []

    # First, try parsing as JSON
    try:
        parsed_json = json.loads(value)
        if isinstance(parsed_json, list):
            for item in parsed_json:
                if 'question' in item and 'answer' in item:
                    jsonqa.append({'question': item['question'], 'answer': item['answer']})
        return jsonqa
    except json.JSONDecodeError:
        # If JSON parsing fails, use regex
        pass

    # Split the string by newlines and parse each line as JSON
    for line in data_string.split('\n'):
        try:
            json_obj = json.loads(line)
            if 'question' in json_obj and 'answer' in json_obj:
                jsonqa.append({'question': json_obj['question'], 'answer': json_obj['answer']})
        except json.JSONDecodeError:
            pass


    # Regex pattern for plain text format
    pattern = r'question:\s*(.*?)\n\s*answer:\s*(.*?)\n\n'
    matches = re.findall(pattern, value, flags=re.DOTALL)
    clean_value = [{"question": m[0].strip(), "answer": m[1].strip()} for m in matches if m[0] and m[1]]
    jsonqa.extend(clean_value)

    return jsonqa

# def get_json_qa(value):
#     jsonqa = []

#     # First, try parsing as JSON
#     try:
#         parsed_json = json.loads(value)
#         if isinstance(parsed_json, dict) and "" in parsed_json and isinstance(parsed_json[""], list):
#             for item in parsed_json[""]:
#                 if 'question' in item and 'answer' in item:
#                     jsonqa.append({'question': item['question'], 'answer': item['answer']})
#             return jsonqa
#     except json.JSONDecodeError:
#         # If JSON parsing fails, use regex
#         pass

#     # Split the string by newlines and parse each line as JSON
#     for line in value.split('\n'):
#         try:
#             json_obj = json.loads(line)
#             if 'question' in json_obj and 'answer' in json_obj:
#                 jsonqa.append({'question': json_obj['question'], 'answer': json_obj['answer']})
#         except json.JSONDecodeError:
#             pass

#     # Regex pattern for plain text format
#     pattern = r'question:\s*(.*?)\n\s*answer:\s*(.*?)\n\n'
#     matches = re.findall(pattern, value, flags=re.DOTALL)
#     clean_value = [{"question": m[0].strip(), "answer": m[1].strip()} for m in matches if m[0] and m[1]]
#     jsonqa.extend(clean_value)

#     return jsonqa

In [30]:
data3['cleaned_qa'].apply(get_json_qa)

NameError: name 'data_string' is not defined

In [27]:
def get_json_qa(value):
    pattern = r'"?question"?:\s*"(.*?)",.*?"?answer"?:\s*"(.*?)"'
    pattern = r'question"?:\s*(.*?)\n\s*answer"?:\s*(.*?)\n\n'
    jsonqa = []

    matches = re.findall(pattern, value, flags=re.DOTALL)
    # Add a conditional filter in the list comprehension
    clean_value = [{"question": m[0], "answer": m[1]} for m in matches if m[0] and m[1] and '"' not in m[0] and '"' not in m[1]]
    jsonqa.append(clean_value)

    jsonqa = list(chain(*jsonqa))

    return jsonqa

In [28]:
import re
pattern = r'"question":\s*"(.*?)",.*?"answer":\s*"(.*?)"'
jsonqa = []

for value in data3['cleaned_qa']:
    matches = re.findall(pattern, value, flags=re.DOTALL)
    # Add a conditional filter in the list comprehension
    clean_value = [{"question": m[0], "answer": m[1]} for m in matches if m[0] and m[1] and '"' not in m[0] and '"' not in m[1]]
    jsonqa.append(clean_value)

jsonqa = list(chain(*jsonqa))

NameError: name 'chain' is not defined

In [ ]:
jsonqa

[{'question': 'What events are happening at SUNY Brockport?',
  'answer': 'You can find out about the events happening at SUNY Brockport by visiting the Be There Events Calendar on the website.'},
 {'question': 'When is the next STEM Day at SUNY Brockport?',
  'answer': 'The next STEM Day at SUNY Brockport is scheduled for February 3, 2024.'},
 {'question': 'Can I schedule a campus tour at SUNY Brockport?',
  'answer': 'Yes, you can schedule an undergraduate campus tour at SUNY Brockport. Tours are available Monday to Friday. Please contact the admissions office for more information.'},
 {'question': 'What programs does SUNY Brockport offer?',
  'answer': 'SUNY Brockport offers over 100 programs including majors, minors, graduate, and online programs. You can explore the full range of programs on the website.'},
 {'question': 'What are the top-ranked facilities at SUNY Brockport?',
  'answer': "SUNY Brockport's Student Union, library resources, athletics programs, and athletic & recrea

In [ ]:
print(json.loads(combined.loc[5, 'questions_x'])['choices'][0]['message']['content'])

1. What opportunities does SUNY Brockport offer for students to develop their leadership skills?
{"question": "What opportunities does SUNY Brockport offer for students to develop their leadership skills?", 
"answer": "SUNY Brockport offers many opportunities for students who are interested in developing their leadership skills."}

2. How can employers connect with the Employer Relations team at SUNY Brockport?
{"question": "How can employers connect with the Employer Relations team at SUNY Brockport?", 
"answer": "To connect with a member of the Employer Relations team, employers can call (585) 395-2159 and schedule a meeting or campus visit."}

3. What is the purpose of the Employee Advisory Board at SUNY Brockport?
{"question": "What is the purpose of the Employee Advisory Board at SUNY Brockport?", 
"answer": "The purpose of the Employee Advisory Board at SUNY Brockport is to provide a forum for employers and staff members to engage in relevant discussions about career and employme

In [ ]:
json.loads(json.loads(combined.loc[5, 'questions_x'])['choices'][0]['message']['content'])

JSONDecodeError: Extra data: line 1 column 2 (char 1)

In [ ]:
import re
import random
from itertools import chain


def generate_json(gpt_output: pd.Series):
    """
    Parses GPT output into a JSON file. This function uses regex to parse for all the instructions (questions), and 
    outputs (answers) from the GPT output which is a dictionary. The parsed data is returned into a Python list of 
    dictionaries, which is appended for each webpage. This list is shuffled to mix up the questions and then dumped 
    to a JSON file.

    Args:
        gpt_output (dict): The dictionary containing the GPT output.
        filename (str): The JSON filename to write the parsed and shuffled list of dictionaries to.
    """

    # The regular expression pattern for a JSON object with "instruction" and "output"
    pattern = r'"question":\s*"(.*?)",.*?"answer":\s*"(.*?)"'
    jsonqa = []

    for value in gpt_output:
        matches = re.findall(pattern, value, flags=re.DOTALL)
        # Add a conditional filter in the list comprehension
        clean_value = [{"question": m[0], "answer": m[1]} for m in matches if m[0] and m[1] and '"' not in m[0] and '"' not in m[1]]
        jsonqa.append(clean_value)

    jsonqa = list(chain(*jsonqa))

    random.shuffle(jsonqa)

    return jsonqa


In [ ]:
test_gpt_output = {"URL": """1. What opportunities does SUNY Brockport offer for students to develop their leadership skills?
{"question": "What opportunities does SUNY Brockport offer for students to develop their leadership skills?", 
"answer": "SUNY Brockport offers many opportunities for students who are interested in developing their leadership skills."}

2. How can employers connect with the Employer Relations team at SUNY Brockport?
{"question": "How can employers connect with the Employer Relations team at SUNY Brockport?", 
"answer": "To connect with a member of the Employer Relations team, employers can call (585) 395-2159 and schedule a meeting or campus visit."}

3. What is the purpose of the Employee Advisory Board at SUNY Brockport?
{"question": "What is the purpose of the Employee Advisory Board at SUNY Brockport?", 
"answer": "The purpose of the Employee Advisory Board at SUNY Brockport is to provide a forum for employers and staff members to engage in relevant discussions about career and employment issues that will contribute to the quality and impact of career programs and services for Brockport students."}

4. How often does the Employee Advisory Board at SUNY Brockport meet and where?
{"question": "How often does the Employee Advisory Board at SUNY Brockport meet and where?", 
"answer": "The Employee Advisory Board at SUNY Brockport meets at least twice each year on the University's campus in Western/Central New York."}

5. How can potential employers benefit from collaborating with the Employer Relations team at SUNY Brockport?
{"question": "How can potential employers benefit from collaborating with the Employer Relations team at SUNY Brockport?", 
"answer": "Potential employers can benefit from collaborating with the Employer Relations team at SUNY Brockport by having their recruiting needs effectively and conveniently met for part-time or full-time jobs, internships, or volunteer opportunities. The team provides exceptional customer service and develops creative solutions to maximize the employer's experience with SUNY Brockport."}"""}

generate_json(test_gpt_output)

[{'question': 'How often does the Employee Advisory Board at SUNY Brockport meet and where?',
  'answer': "The Employee Advisory Board at SUNY Brockport meets at least twice each year on the University's campus in Western/Central New York."},
 {'question': 'What opportunities does SUNY Brockport offer for students to develop their leadership skills?',
  'answer': 'SUNY Brockport offers many opportunities for students who are interested in developing their leadership skills.'},
 {'question': 'How can employers connect with the Employer Relations team at SUNY Brockport?',
  'answer': 'To connect with a member of the Employer Relations team, employers can call (585) 395-2159 and schedule a meeting or campus visit.'},
 {'question': 'What is the purpose of the Employee Advisory Board at SUNY Brockport?',
  'answer': 'The purpose of the Employee Advisory Board at SUNY Brockport is to provide a forum for employers and staff members to engage in relevant discussions about career and employment